In [6]:
import pyspark
import datetime

In [7]:
textfile = sc.textFile("data1.csv")

In [9]:
ventas = textfile.map(lambda x:tuple(x.split(','))).map(lambda x:(datetime.datetime.strptime(x[0],'%d-%m-%Y'),int(x[1]),int(x[2])))

In [10]:
ventas.take(2)

[(datetime.datetime(2017, 1, 1, 0, 0), 118, 1582),
 (datetime.datetime(2019, 9, 9, 0, 0), 107, 894)]

In [11]:
textfile2 = sc.textFile("data2.csv")

In [12]:
productos = textfile2.map(lambda x:tuple(x.split(','))).map(lambda x:(int(x[0]),x[1],x[2]))

In [13]:
productos.take(2)

[(1, 'Descripcion Random', 'Comida'), (2, 'Descripcion Random', 'Muebles')]

## Inciso A

In [14]:
cantidad_ventas = ventas.map(lambda x: (x[1],1))

In [15]:
cantidad_ventas.reduceByKey(lambda x,y:x+y).reduce(lambda x,y: x if x[1]>y[1] else y)[0]

145

## Inciso B

In [16]:
cantidad_ventas.take(2)

[(118, 1), (107, 1)]

In [17]:
categorias_por_id = productos.map(lambda x: (x[0],x[2]))
categorias_por_id.take(2)

[(1, 'Comida'), (2, 'Muebles')]

In [18]:
cantidad_ventas.join(categorias_por_id).map(lambda x:(x[1][1],x[1][0])).reduceByKey(lambda x,y:x+y).reduce(lambda x,y: x if x[1]>y[1] else y)[0]

'Tocador'

## Inciso C

In [19]:
top_vendidos = sc.parallelize(cantidad_ventas.reduceByKey(lambda x,y:x+y).takeOrdered(5,lambda x: -x[1]))

In [20]:
descripcion_por_id = productos.map(lambda x: (x[0],x[1]))

In [21]:
top_vendidos.join(descripcion_por_id).map(lambda x:(x[0],x[1][1],x[1][0])).collect()

[(60, 'Descripcion Random', 5),
 (145, 'Descripcion Random', 7),
 (193, 'Descripcion Random', 6),
 (38, 'Descripcion Random', 5),
 (58, 'Descripcion Random', 7)]

## Inciso D

In [30]:
ventas_ultimo_anio = ventas.filter(lambda x: x[0] >= datetime.datetime(2019,1,1))

In [32]:
#Cambio de claves
ventas_por_producto = ventas_ultimo_anio.map(lambda x: (x[1],1)).reduceByKey(lambda x,y:x+y)

In [36]:
productos_que_vendieron_mas_de_2 = ventas_por_producto.filter(lambda x: x[1]>=2)

In [38]:
ventas_ultimo_anio = ventas_ultimo_anio.map(lambda x: (x[1],x[2]))

In [39]:
productos_que_vendieron_mas_de_2.take(2)

[(160, 2), (116, 2)]

In [40]:
ventas_ultimo_anio.take(2)

[(107, 894), (177, 840)]

In [41]:
productos_que_vendieron_mas_de_2.count()

30

In [42]:
ventas_ultimo_anio.count()

137

In [52]:
ventas_productos_top = ventas_ultimo_anio.join(productos_que_vendieron_mas_de_2)

In [53]:
ventas_productos_top.take(2)

[(160, (512, 2)), (160, (503, 2))]

In [54]:
ventas_productos_top = ventas_productos_top.map(lambda x: (x[0],(x[1][0],x[1][0])))

In [59]:
ventas_productos_top.reduceByKey(lambda x,y: (max(x[0],y[0]),min(x[1],y[1]))).reduce(lambda x,y: x if (x[1][0]-x[1][1] >= y[1][0]-y[1][1]) else y)

(167, (1809, 735))